# Assignment: Image Classification with Transfer Learning

## Setup: Split Flower Photos Dataset

In [ ]:
import pathlib
import shutil
import os

data_dir = pathlib.Path('flower_photos')

if os.path.exists('train_data'):
    shutil.rmtree('train_data')
if os.path.exists('test_data'):
    shutil.rmtree('test_data')
if os.path.exists('validation_data'):
    shutil.rmtree('validation_data')

os.makedirs('train_data', exist_ok=True)
os.makedirs('test_data', exist_ok=True)
os.makedirs('validation_data', exist_ok=True)

for class_folder in data_dir.glob('*'):
    if class_folder.is_dir():
        images = list(class_folder.glob('*.jpg'))
        total = len(images)
        train_split = int(0.7 * total)
        val_split = int(0.15 * total)
        
        os.makedirs(f'train_data/{class_folder.name}', exist_ok=True)
        os.makedirs(f'test_data/{class_folder.name}', exist_ok=True)
        os.makedirs(f'validation_data/{class_folder.name}', exist_ok=True)
        
        for i, img in enumerate(images):
            if i < train_split:
                shutil.copy(img, f'train_data/{class_folder.name}/')
            elif i < train_split + val_split:
                shutil.copy(img, f'validation_data/{class_folder.name}/')
            else:
                shutil.copy(img, f'test_data/{class_folder.name}/')
        
        print(f"{class_folder.name}: {train_split} train, {val_split} val, {total - train_split - val_split} test")

 

daisy: 443 train, 94 val, 96 test
dandelion: 628 train, 134 val, 136 test
roses: 448 train, 96 val, 97 test
sunflowers: 489 train, 104 val, 106 test
tulips: 559 train, 119 val, 121 test
Dataset split complete


## Task 1: Print TensorFlow Version

In [6]:
import tensorflow as tf
print(tf.__version__)

2.20.0


## Task 2: Create test_generator using test_datagen

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'test_data',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

Found 556 images belonging to 5 classes.


## Task 3: Print train_generator Length

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'train_data',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

print(len(train_generator))

Found 2567 images belonging to 5 classes.
129


## Task 4: Print Model Summary

In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 25s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,813,381 (64.14 MB)

 Trainable params: 2,098,693 (8.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

## Task 5: Compile the Model

In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Task 6: Draw Accuracy Curves for extract_feat_model

In [ ]:
import matplotlib.pyplot as plt

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'validation_data',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

extract_feat_model = model

history_extract = extract_feat_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

plt.plot(history_extract.history['accuracy'])
plt.plot(history_extract.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Found 547 images belonging to 5 classes.
Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 167s 1s/step - accuracy: 0.5649 - loss: 1.2154 - val_accuracy: 0.6837 - val_loss: 0.8580
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.6595 - loss: 0.8946 - val_accuracy: 0.7020 - val_loss: 0.7691
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.7004 - loss: 0.8055 - val_accuracy: 0.7221 - val_loss: 0.7595
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - accuracy: 0.7183 - loss: 0.7650 - val_accuracy: 0.7148 - val_loss: 0.7734
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.7164 - loss: 0.7657 - val_accuracy: 0.7294 - val_loss: 0.7300
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.7238 - loss: 0.7079 - val_accuracy: 0.7239 - val_loss: 0.7669
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.7367 - loss: 0.7032 - val_accuracy: 0.7331 - val_loss: 0.7708
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - a

## Task 7: Draw Loss Curves for Fine-tuning Model

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-4]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

plt.plot(history_finetune.history['loss'])
plt.plot(history_finetune.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Task 8: Draw Accuracy Curves for Fine-tuning Model

In [ ]:
plt.plot(history_finetune.history['accuracy'])
plt.plot(history_finetune.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Task 9: Draw Experimental Image (index_to_plot = 1)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

index_to_plot = 1

test_generator.reset()
images, labels = next(test_generator)
predictions = model.predict(images)

img = images[index_to_plot]
true_label = np.argmax(labels[index_to_plot])
pred_label = np.argmax(predictions[index_to_plot])

class_names = list(test_generator.class_indices.keys())

plt.imshow(img)
plt.title(f'True: {class_names[true_label]}, Pred: {class_names[pred_label]}')
plt.axis('off')
plt.show()

## Task 10: Draw Experimental Image (index_to_plot = 1) -


In [ ]:
index_to_plot = 1

test_generator.reset()
images, labels = next(test_generator)
predictions = model.predict(images)

img = images[index_to_plot]
true_label = np.argmax(labels[index_to_plot])
pred_probs = predictions[index_to_plot]
pred_label = np.argmax(pred_probs)

class_names = list(test_generator.class_indices.keys())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.imshow(img)
ax1.set_title(f'True: {class_names[true_label]}\nPred: {class_names[pred_label]}')
ax1.axis('off')

ax2.barh(class_names, pred_probs)
ax2.set_xlabel('Probability')
ax2.set_title('Prediction Probabilities')

plt.tight_layout()
plt.show()